<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Ознакомление-с-исходным-данными" data-toc-modified-id="Ознакомление-с-исходным-данными-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Ознакомление с исходным данными</a></span></li><li><span><a href="#Извлечение-реплик,-где-менеджер-поздоровался" data-toc-modified-id="Извлечение-реплик,-где-менеджер-поздоровался-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Извлечение реплик, где менеджер поздоровался</a></span></li><li><span><a href="#Извлечение-имени-менеджера" data-toc-modified-id="Извлечение-имени-менеджера-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Извлечение имени менеджера</a></span></li><li><span><a href="#Извлечение-имени-компании" data-toc-modified-id="Извлечение-имени-компании-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Извлечение имени компании</a></span></li><li><span><a href="#Извлечение-реплик,-где-менеджер-попрощался" data-toc-modified-id="Извлечение-реплик,-где-менеджер-попрощался-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Извлечение реплик, где менеджер попрощался</a></span></li><li><span><a href="#Проверка-требования-к-менеджеру:-в-каждом-диалоге-должно-быть-прощание" data-toc-modified-id="Проверка-требования-к-менеджеру:-в-каждом-диалоге-должно-быть-прощание-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Проверка требования к менеджеру: в каждом диалоге должно быть прощание</a></span></li><li><span><a href="#Дополнительные-корректировки-таблицы-с-результатами,-вывод-результатов-на-экран" data-toc-modified-id="Дополнительные-корректировки-таблицы-с-результатами,-вывод-результатов-на-экран-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Дополнительные корректировки таблицы с результатами, вывод результатов на экран</a></span></li><li><span><a href="#Заключение" data-toc-modified-id="Заключение-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Заключение</a></span></li></ul></div>

In [ ]:
# закомментированный скрипт для загрузки в виртуальную среду ноутбука библиотеки natasha и натренированного пайплайна библиотеки spacy 
# при необходимости - раскомментировать!

# !pip install natasha
# !python -m spacy download ru_core_news_lg

# Парсинг диалогов

**Цель проекта:** необходимо написать скрипт для парсинга диалогов из файла.

**Исходные данные:** файл с репликами менеджера и клиента.

**Главные задачи, которые должен выполнять скрипт:**
1. Извлекать реплики с приветствием – где менеджер поздоровался.
2. Извлекать реплики, где менеджер представил себя.
3. Извлекать имя менеджера.
4. Извлекать название компании.
5. Извлекать реплики, где менеджер попрощался.
6. Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

**Описание данных:**

* dlg_id — номер диалога
* line_n — номер строки (реплики)
* role — к кому обращена реплика
* text — текст реплики

## Ознакомление с исходным данными

Загрузим необходимые библиотеки и подготовим их к дальнейшей работе

In [1]:
import pandas as pd
import numpy as np
import spacy
import re

In [2]:
from natasha import(
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

Подготовим библиотеку Natasha к дальнейшей работе

In [3]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

Загрузим для анализа русских текстов pipeline, натренированный на русских текстах новостной тематики

In [4]:
nlp = spacy.load("ru_core_news_lg")

Загрузим исходные данные и изучим их

In [5]:
source = pd.read_csv('test_data.csv')

In [6]:
source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [7]:
source.head()

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,manager,Ага


Анализ столбца text показал, что в столбце "role" указана не персона говорящего, а персона к которой обращена реплика. Это видно из стиля общения (наличие приветствия, указание на компанию и т.д.) Таким образом, реплики менеджера содержатся в строках со значением role = client

Создадим копию исходных данных и все изменения будем вносить в нее

In [8]:
data = source.copy()

## Извлечение реплик, где менеджер поздоровался

Для извлечения реплик создадим список шаблонных приветствий

In [9]:
greetings_key_words = ["здравствуйте", "доброе утро", "добрый день", "добрый вечер", "доброго утра", "доброго дня", 
                       "доброго вечера"]

Для создания шаблона (шаблон необходим для библиотеки re) на основе списка приветствий напишем программу

In [10]:
def key_words_to_pattern(key_words_list):
    pattern = ""
    for word in key_words_list:
        pattern += word
        pattern += "|"
    pattern = pattern[:-1]
    return pattern

Получим необходимый шаблон и выведим его на экран

In [11]:
greetings_key_words_pattern = key_words_to_pattern(greetings_key_words)
print(greetings_key_words_pattern)

здравствуйте|доброе утро|добрый день|добрый вечер|доброго утра|доброго дня|доброго вечера


Напишим программу для поиска реплик с приветствиями

In [12]:
def greetings(text):
    answer = False
    text = text.lower()
    pattern = r"{}".format(greetings_key_words_pattern)
    match = re.search(pattern, text)
    if match:
        answer = True
    return answer

Добавим к таблице столбец с данными о наличие приветствия в репликах

In [13]:
data["is_greeting"] = data["text"].apply(greetings)

Для контроля выведим первые пять строк таблицы

In [14]:
data.head()

,dlg_id,line_n,role,text,is_greeting
0,0,0,manager,Алло,False
1,0,1,client,Алло здравствуйте,True
2,0,2,manager,Добрый день,True
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,False
4,0,4,manager,Ага,False


## Извлечение имени менеджера

Протестируем возможности NER библиотек Natasha и spacy для поиска имени менеджера. Тестирование проведем для реплик, содержащихся в исходных данных.

Создадим текст из реплик, содержащих имя менеджера

In [15]:
text_name_test = "Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии \
а мастера мы с вами сотрудничали по видео там. Добрый меня максим зовут компания китобизнес удобно говорить"

Сначала протестируем библиотеку Natasha

In [16]:
doc_test = Doc(text_name_test)
doc_test.segment(segmenter)
doc_test.tag_ner(ner_tagger)
print(doc_test.spans)

[]


Библиотека Natasha выдала пустой список - не нашла ни одного имени. Протестируем NER библиотеки spacy

In [17]:
doc_test = nlp(text_name_test)
for ent in doc_test.ents:
    print(ent.text, ent.label_)

ангелина PER
максим PER


NER библиотеки spacy нашла все имена. Напишем программу для извлечения имени менеджера на основе данной библиотеки. Для того чтобы отсеять те реплики, где менеджер не представился, но в реплике имеется имя (например в виде обращения) введем дополнительное условие: в реплике должен быть токен, лемма которого равняется "звать".

In [18]:
def manager_name(text):
    name = ""
    doc = nlp(text)
    check_phrase = False 
    for token in doc:
        if token.lemma_ == "звать":
            check_phrase=True
    if check_phrase:
        for ent in doc.ents:
            if ent.label_ == "PER":
                name += ent.text
                break
    if len(name) == 0:
        name = "Нет"
    return name

Добавим к таблице столбец с данными о имени менеджера

In [19]:
data["manager_name"] = data["text"].apply(manager_name)

Добавлять к таблице еще один столбец с указанием реплик, где менеджер указал своего имени не нужно, так как если менеджер указал свое имя, то в соответствующей строке будет указано данное имя, если нет, то там будет стоять отметка "Нет"

Для контроля выведим первые пять строк таблицы

In [20]:
data.head()

,dlg_id,line_n,role,text,is_greeting,manager_name
0,0,0,manager,Алло,False,Нет
1,0,1,client,Алло здравствуйте,True,Нет
2,0,2,manager,Добрый день,True,Нет
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,False,ангелина
4,0,4,manager,Ага,False,Нет


## Извлечение имени компании

Протестируем возможности NER разных библиотек на репликах, содержащих имя компании

In [21]:
text_company_test = "Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии. \
Добрый меня максим зовут компания китобизнес удобно говорить. \
Здравствуйте, это компания Российские железные дороги. \
Это компания аэрофлот. \
Это компания Аэрофлот"

Тест для NER библиотеки Natasha

In [22]:
doc_test = Doc(text_company_test)
doc_test.segment(segmenter)
doc_test.tag_ner(ner_tagger)
print(doc_test.spans)

[DocSpan(start=174, stop=200, type='ORG', text='Российские железные дороги', tokens=[...]), DocSpan(start=238, stop=246, type='ORG', text='Аэрофлот', tokens=[...])]


Тест для NER библиотеки Spacy

In [23]:
doc_test = nlp(text_company_test)
for ent in doc_test.ents:
    if ent.label_ != "PER":
        print(ent.text, ent.label_)

Российские железные дороги ORG
аэрофлот ORG
Аэрофлот ORG


Как видно из проведенных тестов, ни одна из библиотек не смогла распознать все названия организаций. При этом библиотека spacy распознала 3 из 5 названий организаций, библиотека Natasha всего одно. С учетом этого в дальнейшем будем использовать библиотеку spacy для поиска названий компаний.

С учетом того, что библиотека spacy не распознала имена "диджитал бизнес" и "китобизнес" помимо поиска при помощи NER библиотеки добавим также дублирующий поиск "ручным методом" по шаблону. "Ручной метод" заключается в поиске в реплике ключевого слова "компания" и проверки следующих после него четырех токенов. Для того чтобы данные токены были включены в название компании они должны удовлетворять шаблону (шаблон company_key_words_pattern - см. ниже) или быть существительными (в именительном или винительном падеже) или прилагательным (при этом название компании не должно заканчиваться прилагательным). Как только обнаружен токен, который не удовлетворяет заданным критериям считаем что названиие компании закончилось и дальше идут токены, не относящиеся к названию компании. Очевидно, что такой поиск не может во всех случаях распознать правильно имя компании и очень сильно зависит от заданного шаблона, поэтому он является дополнительным и призван повысить шансы на распознание имени компании по сравнению с вариантом, когда мы используем только стандартный библиотечный NER.

Зададим шаблон для токенов в имени компании. При задании шаблона были учтены исходные данные. Следует отметить, что исходные данные небольшие по объему (всего 6 диалогов). При увеличении размера исходных данных, можно существенно расширить шаблон, повысив его эффективность

In [24]:
company_key_words = ["ооо", "оао", "диджитал", "лимитид", "интернэшнл", "групп", r"\w+бизнес"]
company_key_words_pattern = key_words_to_pattern(company_key_words)
print(company_key_words_pattern)

ооо|оао|диджитал|лимитид|интернэшнл|групп|\w+бизнес


Напишем программу для извлечения имени компании. Данная программа состоит из двух блоков. Первый блок программы отвечает за поиск имени "ручным" способом (при помощи настраиваемого шаблона). Результат работы данного блока будет записан в отдельный столбец итоговой таблицы "company_name_manual". Блок программы, отвечающий за поиск имени, при помощи NER библиотеки spacy. Результат работы данного блока будет записан в отдельный столбец итоговой таблицы company_name_spacy

In [25]:
def company_name(row):
    text = row[3]
    doc = nlp(text)
    # блок программы, отвечающий за поиск имени "ручным" способом (при помощи настраиваемого шаблона)
    # результат работы данного блока будет записан в отдельный столбец итоговой таблицы "company_name_manual"
    name = ""
    doc_lenth = len(doc)
    position = np.nan
    for token in doc:
        if token.text.lower() == 'компания':
            position = token.i
            break
    if position == np.nan or position == doc_lenth - 1:
        row["company_name_manual"] = "Нет"
    else:
        if token.i + 5 > doc_lenth - 1:
            stop = doc_lenth
        else:
            stop = token.i + 5
        for i in range(token.i + 1, stop):
            match = re.search(r"{}".format(company_key_words_pattern), doc[i].text.lower())
            if match:
                name += " "
                name += doc[i].text
                continue
            if doc[i].pos_ == "ADJ" and i != stop - 1:
                name += " "
                name += doc[i].text
                continue
            if (doc[i].pos_ == "NOUN") and (doc[i].morph.get("Case")[0] == "Acc" or doc[i].morph.get("Case")[0] == "Nom"):
                name += " "
                name += doc[i].text
            else:
                break
        if len(name) != 0:
            name = name[1:]
            doc_last_name = nlp(name.split()[-1])
            if doc_last_name[0].pos_ == "ADJ":
                name = " ".join(name.split()[:-1])
        else:
            name = "Нет"
        row["company_name_manual"] = name
    # блок программы, отвечающий за поиск имени при помощи NER библиотеки spacy
    name_spacy = ""
    for ent in doc.ents:
        if ent.label_ == "ORG":
            name_spacy += ent.text
            break
    if len(name_spacy) == 0:
        name_spacy = "Нет"
    row["company_name_spacy"] = name_spacy
    return row

Добавим к таблице столбцы с именем компании ("company_name_manual", "company_name_spacy"). Для контроля выведим первые пять строк таблицы

In [26]:
data = data.apply(company_name, axis=1)
data.head()

,dlg_id,line_n,role,text,is_greeting,manager_name,company_name_manual,company_name_spacy
0,0,0,manager,Алло,False,Нет,Нет,Нет
1,0,1,client,Алло здравствуйте,True,Нет,Нет,Нет
2,0,2,manager,Добрый день,True,Нет,Нет,Нет
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,False,ангелина,диджитал бизнес,Нет
4,0,4,manager,Ага,False,Нет,Нет,Нет


## Извлечение реплик, где менеджер попрощался

Для извлечения реплик создадим шаблон прощаний и выведим его на экран

In [27]:
farewell_key_words = ["до свидания", "всего хорошего", "всего доброго", "всего наилучшего", "удачи"]
farewell_key_words_pattern = key_words_to_pattern(farewell_key_words)
print(farewell_key_words_pattern)

до свидания|всего хорошего|всего доброго|всего наилучшего|удачи


Напишим программу для поиска реплик с прощаниями

In [28]:
def farewells(text):
    answer = False
    text = text.lower()
    pattern = r"{}".format(farewell_key_words_pattern)
    match = re.search(pattern, text)
    if match:
        answer = True
    return answer

Добавим к таблице столбец с данными о наличие приветствия в репликах

In [29]:
data["is_farewell"] = data["text"].apply(farewells)

Для контроля выведим первые пять строк таблицы

In [30]:
data.head()

,dlg_id,line_n,role,text,is_greeting,manager_name,company_name_manual,company_name_spacy,is_farewell
0,0,0,manager,Алло,False,Нет,Нет,Нет,False
1,0,1,client,Алло здравствуйте,True,Нет,Нет,Нет,False
2,0,2,manager,Добрый день,True,Нет,Нет,Нет,False
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,False,ангелина,диджитал бизнес,Нет,False
4,0,4,manager,Ага,False,Нет,Нет,Нет,False


## Проверка требования к менеджеру: в каждом диалоге должно быть прощание

Фактически проверка наличия прощания менеджера выполнена в разделе 5, где у каждой реплики есть отметка о том, есть ли прощание или нет. Однако данный формат представления результатов не совсем удобен: необходимо просмотреть все строки. Создадим под данную задачу отдельную таблицу. Для этого сначала оставим только те реплики, которые адресованы менеджером клиенту (отметка role = "client") и только два столбца: "dlg_id" и "is_farewell". Затем выполним группировку таблицы по номеру диалога (столбец "dlg_id"). При группировке будем считать в столбце is_farewell сумму (True = 1).

In [31]:
data_dialog_farawell_check = data[data["role"] == "client"][["dlg_id", "is_farewell"]]
data_dialog_farawell_check = data_dialog_farawell_check.groupby("dlg_id").sum()
data_dialog_farawell_check

,is_farewell
dlg_id,
0,1
1,2
2,0
3,1
4,1
5,1


Если в столбце "is_farewell" значение равно 0, следовательно, менеджер не попрощался, если значение равно или больше 1 - менеджер попрощался. Учтем это, внеся соответствующую замену в таблицу

In [32]:
data_dialog_farawell_check["is_farewell"] = data_dialog_farawell_check["is_farewell"].apply(lambda x: True if x > 0 else False)
data_dialog_farawell_check

,is_farewell
dlg_id,
0,True
1,True
2,False
3,True
4,True
5,True


Как видно из таблицы, в 5 диалогах из 6 менеджер попрощался

## Дополнительные корректировки таблицы с результатами, вывод результатов на экран

Все поставленные задачи к скрипту относятся к менеджеру. Таким образом, в таблице можно удалить лишние строки, где реплики обращены от клиента к менеджеру (role = manager). Это существенно сократит размер таблицы. Для ускорения парсинга рационально данное сокращение таблицы выполнять непосредственно перед работой скрипта. Однако в данном ноутбуке (разделы 1-6) этого (удаления лишних реплик) не делалось с учетом того, что в задании было указано, что исходные данные изменять нельзя.

In [33]:
data_short_version = data[data["role"] == "client"]

Выведим на экран таблицу data_short_version

In [34]:
pd.options.display.max_rows = 300
data_short_version

,dlg_id,line_n,role,text,is_greeting,manager_name,company_name_manual,company_name_spacy,is_farewell
1,0,1,client,Алло здравствуйте,True,Нет,Нет,Нет,False
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,False,ангелина,диджитал бизнес,Нет,False
5,0,5,client,Угу ну возможно вы рассмотрите и другие вариан...,False,Нет,Нет,Нет,False
8,0,8,client,Угу а на что вы обращаете внимание при выборе,False,Нет,Нет,Нет,False
11,0,11,client,Что для вас приоритет,False,Нет,Нет,Нет,False
15,0,15,client,Ну у вас срок заканчивается поэтому мы набрали...,False,Нет,Нет,Нет,False
29,0,29,client,А так нет не только поэтому просто я обратила ...,False,Нет,Нет,Нет,False
34,0,34,client,А если вы 19 являетесь то лучше то идти бесплатно,False,Нет,Нет,Нет,False
36,0,36,client,Ага хорошо,False,Нет,Нет,Нет,False
45,0,45,client,Индивидуальным поэтому не все то есть сотрудни...,False,Нет,Нет,Нет,False


## Заключение

1. В результате исполнения скрипта в ноутбуке мы имеем две версии таблицы. Первая основная называется data. Она содержит все реплики, в том числе обращенные от клиента к менеджеру (скрипт, представленный в данном ноутбуке, применялся ко всем репликам). Вторая версия таблицы называется data_short_version и является сокращенной версией таблицы data. Сокращение достигнуто путем удаления лишних строк, где реплики направлены от клиента к менеджеру. В разделе 7 ноутбука для удобства анализа работы скрипта на экран полностью выведена таблица data_short_version.
2. Последняя задача (проверить наличие прощания в каждом диалоге) несколько выбивается от остальных, так как необходимо анализировать не отдельную реплику, а диалог целиком. Для выполнения данной задачи требуется группировка таблицы по столбцу с номером диалога. Результат решения данной задачи представлен в отдельной таблице data_dialog_farawell_check (см. раздел 6 ноутбука)
3. Самой сложной задачей, вероятно, явлется NER для имени компании. Ввиду этого в скрипте заложено два дублирующих друг друга решений. Одно основано на NER библиотеки spacy, другое является ручным и основано на поиске в реплике ключевого слова "компания" и анализе идущих непосредственно за ним токенов.
4. Часть решений для парсинга основана на шаблонах (выявление синонимичных выражений, ключевых слов, фраз). Эффективность данных шаблонов, вероятно, может быть существенно повышена в случае увеличения объема исходных данных: текущие исходные данные включают в себя только 6 диалогов.